In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))
os.environ["OMP_NUM_THREADS"] = '1'

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer.trainer_twoHeaded import Trainer_twoHeaded
from src.trainer.trainer_VAE import Trainer_VAE
from src.models.classifiers import *
from src.trainer.model_class import Model_class
from src.trainer.loss_class import *

from src.models.autoencoders import *
from src.models.joinedModel import *
from src.models.twoHeaded import *

from src.tools import upsampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 1024
INIT_PARAM = 512
NUM_FEATURES = 50
LATENT_REPR = 5
GAMMA = 0.35

In [3]:
df = pd.read_csv('../data/data_fe.csv')
feature_importance = pd.read_csv('../data/feature_importance/feature_imporstance_WS.csv')

In [4]:
features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

df.to_csv('../data/df_to_enc.csv', index = False)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'], random_state=42,
                                                 train_size=0.7)

In [6]:
X_train, y_train = upsampling(GAMMA, X_train, y_train) # 0.3

In [7]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis = 1)

train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [8]:
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [9]:
model = TwoHeadedModel(Baseline_classifier(NUM_FEATURES, INIT_PARAM).classifier,
                       Simple_classifier(LATENT_REPR).classifier)

In [10]:
loss = Loss_class_TH(TwoHeadedLoss(2,FocalLoss(gamma=2),nn.MSELoss()))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.90)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'fe_{INIT_PARAM}_{BATCH_SIZE}_{NUM_FEATURES}_BASELINE',
    project="Internship_project",
    dir = '../logs/'
)

In [11]:
trainer = Trainer_twoHeaded(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/classifier/',
                  saving_model=False
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train_model(learning_params)
wandb.finish()

I'm studying hard now🧐, don't disturb!:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  10%|█         | 1/10 [00:02<00:24,  2.75s/it]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  20%|██        | 2/10 [00:03<00:10,  1.29s/it]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm st

Epoch: 1 of 40, 0.108 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  10%|█         | 1/10 [00:00<00:03,  2.89it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  20%|██        | 2/10 [00:00<00:02,  3.10it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm st

Epoch: 2 of 40, 0.065 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  10%|█         | 1/10 [00:00<00:02,  3.72it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  20%|██        | 2/10 [00:00<00:02,  3.67it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm st

...

Epoch: 25 of 40, 0.066 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  10%|█         | 1/10 [00:00<00:02,  3.09it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  20%|██        | 2/10 [00:00<00:02,  3.14it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm st

I'm studying hard now🧐, don't disturb!:  80%|████████  | 8/10 [00:02<00:00,  3.18it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  90%|█████████ | 9/10 [00:03<00:00,  2.94it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Let's see how good I am...:   0%|          | 0/3 [00:00<?, ?it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Let's see how good 

Epoch: 26 of 40, 0.076 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  10%|█         | 1/10 [00:00<00:03,  2.59it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm studying hard now🧐, don't disturb!:  20%|██        | 2/10 [00:00<00:03,  2.55it/s]c:\Users\dimaf\miniconda3\envs\torch\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
I'm st